# 🏥 Histopathologic Cancer Detection

## 📝 Problem Description
The challenge is a binary image classification problem aimed at identifying metastatic cancer in small image patches taken from larger digital pathology scans. This competition, hosted on Kaggle, involves creating models to accurately detect the presence of tumor tissue in 32x32 pixel image patches. Early and precise detection of metastatic cancer is crucial for effective patient treatment and management.

## 📊 Data Description
The dataset for this competition is a modified version of the PatchCamelyon (PCam) benchmark dataset. It contains 96x96 pixel image patches extracted from larger whole-slide images, with each patch labeled as either containing metastatic tissue (positive class) or not (negative class). The dataset is divided into training and test sets, where the training set includes both the images and their labels. The structure and size of the dataset facilitate the training of convolutional neural networks (CNNs) for image classification tasks. The dataset can be accessed from [Kaggle](https://www.kaggle.com/competitions/histopathologic-cancer-detection).

In [5]:
# Here we can see our Data's Labels for the training data
import pandas as pd

train_df = pd.read_csv("./train/_labels.csv")
train_df

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0
...,...,...
220020,53e9aa9d46e720bf3c6a7528d1fca3ba6e2e49f6,0
220021,d4b854fe38b07fe2831ad73892b3cec877689576,1
220022,3d046cead1a2a5cbe00b2b4847cfb7ba7cf5fe75,0
220023,f129691c13433f66e1e0671ff1fe80944816f5a2,0


## 🔗 Reference
Veeling, B. S., Linmans, J., Winkens, J., Cohen, T., & Welling, M. (2018). Rotation Equivariant CNNs for Digital Pathology. arXiv:1806.03962.